In [1]:
#聲音處理套件
import librosa
import librosa.display
#資料處理套件
import pandas as pd
import numpy as np
import torch

In [2]:
#音訊資訊CSV檔位置
CSV_FILE_PATH = "C:/Users/Jia-Wei Chang/Desktop/dataset/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "C:/Users/Jia-Wei Chang/Desktop/dataset/ESC-50-master/ESC-50-master/audio/"

In [3]:
#讀取CSV檔
df = pd.read_csv(CSV_FILE_PATH)
df.head()

#篩選出ESC10作為訓練資料
df_10 = df[df['esc10']==True]
#刪去無用欄位
df_10 = df_10.drop(['esc10','src_file','take'], axis=1)

#取出類別，去除重複並排序
classes = df_10['category'].unique()
class_dict = {i:x for x,i in enumerate(classes)}
print(class_dict)

#重新標label
df_10['target'] = df_10['category'].map(class_dict)

{'dog': 0, 'chainsaw': 1, 'crackling_fire': 2, 'helicopter': 3, 'rain': 4, 'crying_baby': 5, 'clock_tick': 6, 'sneezing': 7, 'rooster': 8, 'sea_waves': 9}


In [4]:
class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sr,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

In [5]:
#ESC資料集聲音處理
def trainPreprocess(fold, soundLength):
    #聲音長度
    soundLength = soundLength
    #一個聲音切成幾段
    soundCount = 6 - soundLength
    
    #要回傳的資料 
    #資料 標籤 是否為電鋸聲/正常ESC10標籤
    x_train, y_train_2, y_train_10 = [], [], []
    x_val, y_val_2, y_val_10 = [], [], []
    
    #根據fold來切分train/val
    train_df = df_10[df_10.fold != fold]
    val_df = df_10[df_10.fold == fold]
    
    #轉換train資料/添加標籤
    for data in train_df.itertuples():
        sig , sr = librosa.load(DATA_PATH+data[1], sr=conf.sr)
        for i in range(soundCount):
            sig_ = sig[i*sr : int((i+soundLength)*sr)]
            mel_spec = audio_to_melspectrogram(conf, sig_)
            x_train.append(mel_spec)
            #正常標籤
            y_train_10.append(data[3])
            #在ESC-10中 電鋸聲標籤為1
            if(data[3]==1):
                y_train_2.append(1)
            else:
                y_train_2.append(0)
            
    #轉換val資料/添加標籤
    for data in val_df.itertuples():
        sig , sr = librosa.load(DATA_PATH+data[1], sr=conf.sr)
        for i in range(soundCount):
            sig_ = sig[i*sr : int((i+soundLength)*sr)]
            mel_spec = audio_to_melspectrogram(conf, sig_)
            x_val.append(mel_spec)
            #正常標籤
            y_val_10.append(data[3])
            #在ESC-10中 電鋸聲標籤為1
            if(data[3]==1):
                y_val_2.append(1)
            else:
                y_val_2.append(0)

          
    #通通轉成 torch 的 tensor 形式
    x_train = torch.Tensor(x_train)
    y_train_10 = torch.Tensor(y_train_10).type(torch.LongTensor)
    y_train_2 = torch.Tensor(y_train_2).type(torch.LongTensor)
    x_val = torch.Tensor(x_val) 
    y_val_10 = torch.Tensor(y_val_10).type(torch.LongTensor)
    y_val_2 = torch.Tensor(y_val_2).type(torch.LongTensor)
    
    #轉維度 BCHW
    x_train = torch.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1], x_train.shape[2]))
    x_val = torch.reshape(x_val, (x_val.shape[0], 1, x_val.shape[1], x_val.shape[2]))
    
    ESC_DATA_PATH = "C:/Users/Jia-Wei Chang/hu_file/ESC20220524_pytorch/Preprocessing/ESC/Fold"
    x_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_train.pt"
    y_train_10_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train_10.pt" 
    y_train_2_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train_2.pt" 
    x_val_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_val.pt" 
    y_val_10_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_val_10.pt" 
    y_val_2_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_val_2.pt" 
    
    torch.save(x_train, x_train_path)
    torch.save(y_train_10, y_train_10_path)
    torch.save(y_train_2, y_train_2_path)
    torch.save(x_val, x_val_path)
    torch.save(y_val_10, y_val_10_path)
    torch.save(y_val_2, y_val_2_path)

In [6]:
for i in range(1,6):
    for s in range(1,6):
        trainPreprocess(i,s)

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-6.10351562e-05 -9.15527344e-05 -3.05175781e-05 ... -1.48925781e-01
 -1.42639160e-01 -1.23687744e-01] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.09457397 -0.06793213 -0.03421021 ...  0.0128479  -0.01296997
 -0.03106689] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.05484009 -0.07583618 -0.08630371 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,


C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.22399902 -0.2116394  -0.19573975 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00680542 -0.00534058
 -0.00048828] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.00738525 0.0177002  0.0295105  ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.03991699  0.01028442  0.1065979  ... -0.00128174 -0.00146484
 -0.00137329] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-1.1596680e-03 -1.4343262e-03 -1.6784668e-03 ...  6.1035156e-05
  6.1035156e-05  6.1035156e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[6.1035156e-05 6.1035156e-05 6.1035156e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspect

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 0.03240967 -0.0090332   0.00115967 ...  0.01864624 -0.00408936
 -0.01293945] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.01739502 -0.02114868 -0.00866699 ... -0.01019287 -0.00793457
 -0.01467896] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 0.06311035  0.08270264  0.06872559 ... -0.01724243 -0.02059937
 -0.02203369] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.00360107 -0.00366211 -0.00396729 ...  0.0090332   0.00897217
  0.0085144 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 8.0261230e-03  7.8735352e-03  8.0261230e-03 ... -3.0517578e-04
 -2.1362305e-04 -9.1552734e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 0.0000000e+00  3.0517578e-05  3.0517578e-05 ... -4.0588379e-03
 -4.1809082e-03 -4.1503906e-03] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.me

...

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.01828003 -0.0239563  -0.02160645 ...  0.00909424  0.00946045
  0.00799561] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.04522705 0.05926514 0.0513916  ... 0.04656982 0.06063843 0.0508728 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.43371582 0.4451599  0.45394897 ... 0.06167603 0.07498169 0.06008911] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.05276489 -0.03640747 -0.00112915 ...  0.04107666  0.02893066
  0.01260376] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-3.0517578e-05 -6.1035156e-05 -6.1035156e-05 ...  4.4860840e-03
  4.8522949e-03  3.0822754e-03] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.01000977 0.01074219 0.00479126 ... 0.0375061  0.0718689  0.07342529] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Us

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 0.00305176  0.00268555  0.00131226 ... -0.01660156 -0.00024414
  0.00848389] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.00036621 0.00064087 0.00082397 ... 0.00131226 0.00183105 0.00082397] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.00628662 -0.01412964 -0.03182983 ... -0.00222778 -0.00161743
 -0.00042725] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.05505371 -0.07888794 -0.08105469 ... -0.01754761 -0.00860596
 -0.00222778] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.00302124 0.02703857 0.034729   ... 0.08093262 0.06768799 0.04376221] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.00045776 -0.00048828 -0.00048828 ... -0.08312988 -0.09207153
 -0.07217407] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\

C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[0.00521851 0.00756836 0.01034546 ... 0.08010864 0.08148193 0.05535889] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[-0.0345459  -0.04321289 -0.02978516 ...  0.00836182 -0.00015259
 -0.00097656] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\AppData\Local\Temp\ipykernel_15324\1963153878.py:11: FutureWarning: Pass y=[ 0.00210571  0.02444458  0.04382324 ... -0.13647461 -0.18185425
 -0.14331055] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectrogram = librosa.feature.melspectrogram(audio,
C:\Users\Jia-Wei Chang\